## Exemple

Prenons l'exemple de la base de données de e-commerce suivante avec seuil = 2:

|Client	| Séquences d'achat |
|---|---|
|1	| Téléphone -> Écouteurs -> Batterie|
|2	| Ordinateur -> Sac -> Souris|
|3	| Téléphone -> Coque|
|4 | Écouteurs -> Batterie|
|5	| Téléphone -> Écouteurs|
|6 | Ordinateur -> Moniteur|
|7	| Téléphone -> Écouteurs -> Batterie|
|8	| Sac -> Souris|

**Représentation verticale:**

| Article | Séquences |
|---|---|
| Téléphone | 1, 3, 5, 7 |
| Écouteurs | 1, 4, 5, 7 |
| Batterie | 1, 4, 7 |
| Ordinateur | 2, 6 |
| Sac | 2, 8 |
| Souris | 2, 8 |
| Coque | 3 |

**Création de classes d'équivalence:**

* Initialement, chaque article est dans sa propre classe. On obtient les classes d'equivalence suivantes :
    * Classe 1 : Telephone
    * Classe 1 : Ecouteurs
    * Classe 1 : Batterie
    * Classe 1 : Sac
    ...
    
* les candidats de longueur 1 seraient :
    * (Telephone)
    * (Ecouteurs)
    * (Batterie)
    * (Sac)
...

**Elagage:

* En parcourant les séquences, on fusionne les classes de "Téléphone" et "Coque" car elles apparaissent ensemble dans la séquence 3. On fait de meme pour toutes les classes qui apparaissaient ensemble dans la meme sequence. On obtient donc les classes d'équivalence suivantes :
    * Classe 1 : Téléphone, Coque
    * Classe 2 : Écouteurs, Batterie
    ...
    * Classe 3 : Ordinateur, Sac, Souris
    * Classe 3 : Téléphone, Écouteurs, Batterie

* les candidats de longueur 2 seraient :
    * (Téléphone, Écouteurs)
    * (Téléphone, Batterie)
    * (Telephone, Coque)
    * (Coque, Batterie)
    * (Ordinateur, Sac)
    * (Ordinateur, Souris)
    * (Sac, Souris)
    ...

**Avec un seul de 25%, les Candidats de longueur 2:

La paire (Téléphone, Écouteurs) apparaît dans les séquences 1, 5 et 7.
Le support est donc de 3/8 = 37.5%.

La paire (Ordinateur, Sac) apparaît dans les séquences 2 et 8.
Le support est donc de 2/8 = 25%.

La paire (Ecouteur, Batterie) apparaît dans les séquences 1, 4 et 7.
Le support est donc de 3/8 = 37.5%.

La paire (Téléphone, Coque) apparaît dans la séquence 3.
Le support est donc de 1/8 = 12.5%.


(Téléphone, Écouteurs): Support = 3/8 = 37.5% (Fréquent)
(Écouteurs, Batterie): Support = 3/8 = 25% (Fréquent)
(Ordinateur, Sac): Support = 2/8 = 25% (Fréquent)
(Téléphone, Coque): Support = 1/8 = 12.5% (Non fréquent)
... 

* En parcourant les séquences, on fusionne les classes de "Téléphone","Ecouteur" et "Batterie" car elles apparaissent ensemble dans la séquence. On fait de meme pour toutes les classes qui apparaissaient ensemble dans la meme sequence. On obtient donc les classes d'équivalence suivantes :

**les candidats de longueur 3 seraient :

(Ordinateur, Sac, Souris)
(Téléphone, Écouteurs, Batterie)
...

**Avec un seul de 25%, les Candidats de longueur 3:

(Téléphone, Écouteurs, Batterie): Support = 2/8 = 25% (Frequent)
(Ordinateur, Sac, Souris): Support = 1/8 = 12,5% (Non Frequent)
...

## Ce qu'il faut retenir de l'algorithme SPADE
### 1- Representation verticale
### 2- Generation des classes d'equivalences qui seront les candidats
### 3- Prunning
### 4- Elagage
### 5- Iteration des etapes 2 a 4 jusqu'a ce qu'on obtienne plus de super classe

In [1]:
import random
import pandas as pd
from collections import defaultdict


In [16]:
"""# Générer des séquences aléatoires
items = ["Telephone", "Ordinateur", "Ecouteurs", "Batterie", "Cle-USB", "Souris", "Clavier", "Sac", "Disque-dur"]
num_sequences = 20
max_length = 7

sequences = []
for _ in range(num_sequences):
    length = random.randint(3, max_length)
    sequence = [(item, tid + 1) for tid, item in enumerate(random.sample(items, length))]
    sequences.append(sequence)

# Convertir les séquences en DataFrame
df = pd.DataFrame([{"Sequence": i + 1, "Item": item, "ID": tid} for i, seq in enumerate(sequences) for item, tid in seq])

# Afficher le DataFrame
print("DataFrame:")
df.head(50)"""

'# Générer des séquences aléatoires\nitems = ["Telephone", "Ordinateur", "Ecouteurs", "Batterie", "Cle-USB", "Souris", "Clavier", "Sac", "Disque-dur"]\nnum_sequences = 20\nmax_length = 7\n\nsequences = []\nfor _ in range(num_sequences):\n    length = random.randint(3, max_length)\n    sequence = [(item, tid + 1) for tid, item in enumerate(random.sample(items, length))]\n    sequences.append(sequence)\n\n# Convertir les séquences en DataFrame\ndf = pd.DataFrame([{"Sequence": i + 1, "Item": item, "ID": tid} for i, seq in enumerate(sequences) for item, tid in seq])\n\n# Afficher le DataFrame\nprint("DataFrame:")\ndf.head(50)'

In [3]:
df = pd.read_csv("store_data.csv")
df.head(50)

,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,whole wheat pasta,french fries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,soup,light cream,shallot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,frozen vegetables,spaghetti,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,french fries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,eggs,pet food,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Convertir le DataFrame en dictionnaire de séquences
sequences_dict = defaultdict(list)
for idx, row in df.iterrows():
    for col in df.columns:
        if pd.notna(row[col]):
            sequences_dict[idx].append((row[col], idx + 1))


In [5]:
# Initialiser la liste pour les données transformées
transformed_data = []

# Parcourir le dictionnaire de séquences et ajouter les données à la liste
for seq, items in sequences_dict.items():
    for item, id_ in items:
        transformed_data.append([seq, item, id_])

# Créer le DataFrame final
df = pd.DataFrame(transformed_data, columns=['Sequence', 'Item', 'ID'])

# Afficher le DataFrame transformé
df.head(50)


,Sequence,Item,ID
0,0,burgers,1
1,0,meatballs,1
2,0,eggs,1
3,1,chutney,2
4,2,turkey,3
5,2,avocado,3
6,3,mineral water,4
7,3,milk,4
8,3,energy bar,4
9,3,whole wheat rice,4


In [6]:
# Convertir le DataFrame en dictionnaire de séquences
sequences_dict = defaultdict(list)
for _, row in df.iterrows():
    sequences_dict[row['Sequence']].append((row['Item'], row['ID']))


In [14]:
# Représentation verticale
vertical_db = defaultdict(list)
for _, row in df.iterrows():
    vertical_db[row['Item']].append(row['Sequence'])

print("\nReprésentation verticale :")
for item, sequences in vertical_db.items():
    print(f"{item}: {sequences}")

# Convertir vertical_db en une liste de tuples
data_vertical = [(item, seq) for item, sequences in vertical_db.items() for seq in sequences]

# Créer le DataFrame vertical
df_vertical = pd.DataFrame(data_vertical, columns=['Item', 'Sequence'])

print("\nDataFrame vertical :")
df_vertical



Représentation verticale :
burgers: [0, 11, 22, 48, 70, 80, 82, 88, 90, 93, 100, 101, 110, 117, 124, 132, 149, 155, 157, 164, 172, 180, 197, 223, 225, 245, 247, 253, 269, 300, 304, 316, 334, 395, 414, 438, 445, 455, 488, 507, 512, 517, 524, 551, 552, 578, 581, 584, 594, 617, 641, 666, 681, 690, 693, 697, 698, 722, 742, 743, 772, 780, 799, 809, 830, 846, 852, 871, 878, 880, 897, 904, 907, 935, 936, 955, 957, 972, 974, 988, 1004, 1015, 1016, 1020, 1030, 1040, 1043, 1063, 1073, 1096, 1100, 1104, 1138, 1148, 1158, 1182, 1203, 1230, 1236, 1262, 1283, 1288, 1294, 1305, 1309, 1313, 1316, 1341, 1362, 1367, 1388, 1391, 1392, 1408, 1409, 1412, 1413, 1415, 1419, 1424, 1427, 1464, 1484, 1486, 1489, 1499, 1503, 1510, 1526, 1549, 1553, 1560, 1570, 1614, 1622, 1626, 1639, 1653, 1681, 1689, 1706, 1739, 1744, 1764, 1768, 1772, 1773, 1774, 1787, 1819, 1821, 1829, 1855, 1863, 1865, 1878, 1894, 1897, 1913, 1914, 1918, 1922, 1939, 1954, 1969, 1976, 1996, 2022, 2023, 2031, 2034, 2045, 2054, 2067, 2071, 208

,Item,Sequence
0,burgers,0
1,burgers,11
2,burgers,22
3,burgers,48
4,burgers,70
...,...,...
29338,whole weat flour,6387
29339,whole weat flour,6394
29340,whole weat flour,6959
29341,whole weat flour,7069


In [7]:
sequences_dict

defaultdict(list,
            {0: [('burgers', 1), ('meatballs', 1), ('eggs', 1)],
             1: [('chutney', 2)],
             2: [('turkey', 3), ('avocado', 3)],
             3: [('mineral water', 4),
              ('milk', 4),
              ('energy bar', 4),
              ('whole wheat rice', 4),
              ('green tea', 4)],
             4: [('low fat yogurt', 5)],
             5: [('whole wheat pasta', 6), ('french fries', 6)],
             6: [('soup', 7), ('light cream', 7), ('shallot', 7)],
             7: [('frozen vegetables', 8), ('spaghetti', 8), ('green tea', 8)],
             8: [('french fries', 9)],
             9: [('eggs', 10), ('pet food', 10)],
             10: [('cookies', 11)],
             11: [('turkey', 12),
              ('burgers', 12),
              ('mineral water', 12),
              ('eggs', 12),
              ('cooking oil', 12)],
             12: [('spaghetti', 13), ('champagne', 13), ('cookies', 13)],
             13: [('mineral water', 14), ('sa

In [8]:
# Convertir le dictionnaire en une liste de tuples pour pouvoir le trier et le slicer
sequences_list = list(sequences_dict.items())

# Sélectionner les premières lignes du dictionnaires pour tester l'algorithmes
first_sequences = sequences_list[:500]

# Convertir la sélection de retour en un dictionnaire
sequences_dict_first = dict(first_sequences)

# Afficher les 200 premières lignes pour vérification
print(sequences_dict_first)


{0: [('burgers', 1), ('meatballs', 1), ('eggs', 1)], 1: [('chutney', 2)], 2: [('turkey', 3), ('avocado', 3)], 3: [('mineral water', 4), ('milk', 4), ('energy bar', 4), ('whole wheat rice', 4), ('green tea', 4)], 4: [('low fat yogurt', 5)], 5: [('whole wheat pasta', 6), ('french fries', 6)], 6: [('soup', 7), ('light cream', 7), ('shallot', 7)], 7: [('frozen vegetables', 8), ('spaghetti', 8), ('green tea', 8)], 8: [('french fries', 9)], 9: [('eggs', 10), ('pet food', 10)], 10: [('cookies', 11)], 11: [('turkey', 12), ('burgers', 12), ('mineral water', 12), ('eggs', 12), ('cooking oil', 12)], 12: [('spaghetti', 13), ('champagne', 13), ('cookies', 13)], 13: [('mineral water', 14), ('salmon', 14)], 14: [('mineral water', 15)], 15: [('shrimp', 16), ('chocolate', 16), ('chicken', 16), ('honey', 16), ('oil', 16), ('cooking oil', 16), ('low fat yogurt', 16)], 16: [('turkey', 17), ('eggs', 17)], 17: [('turkey', 18), ('fresh tuna', 18), ('tomatoes', 18), ('spaghetti', 18), ('mineral water', 18), (

In [9]:
# Fonction pour vérifier si un motif est une sous-séquence d'une transaction
def is_subsequence(subseq, seq):
    it = iter(seq)
    for item in subseq:
        found = False
        for trans_item in it:
            trans_item_set = set([trans_item[0]])  # Prendre uniquement l'article et non l'ID
            # Vérifiez si item est une séquence ou un élément unique
            if isinstance(item, tuple) or isinstance(item, list):
                # Si item est un tuple ou une liste, comparez comme un ensemble
                if set(item).issubset(trans_item_set):
                    found = True
                    break
            else:
                # Si item est un élément unique, comparez directement
                if {item}.issubset(trans_item_set):
                    found = True
                    break
        if not found:
            return False
    return True


In [10]:
# Générer les motifs fréquents d'un seul élément
def get_single_item_frequent_patterns(sequences, min_support):
    item_support = defaultdict(int)
    for sid, sequence in sequences.items():
        unique_items_in_seq = set()
        for itemset in sequence:
            for item in itemset:
                unique_items_in_seq.add(item)
        for item in unique_items_in_seq:
            item_support[(item,)] += 1
    frequent_patterns = {pattern: support for pattern, support in item_support.items() if support >= min_support}
    return frequent_patterns


In [11]:
# Générer des candidats à partir de motifs fréquents précédents
def generate_candidates(prev_frequent_patterns):
    candidates = set()
    patterns = list(prev_frequent_patterns.keys())
    for i in range(len(patterns)):
        for j in range(i, len(patterns)):
            p1, p2 = patterns[i], patterns[j]
            # Combiner les motifs fréquents pour créer des motifs séquentiels plus longs
            new_pattern = p1 + p2
            candidates.add(new_pattern)
    return candidates


In [12]:
# Calculer le support des candidats
def count_support(sequences, candidates):
    support_count = defaultdict(int)
    for candidate in candidates:
        for sid, sequence in sequences.items():
            if is_subsequence(candidate, sequence):
                support_count[candidate] += 1
    return support_count


In [13]:
# Algorithme SPADE
def spade_algorithm(sequences, min_support):
    # Obtenir les motifs fréquents d'un seul élément
    frequent_patterns = get_single_item_frequent_patterns(sequences, min_support)
    all_frequent_patterns = frequent_patterns.copy()

    # Étendre les motifs d'un seul élément en motifs plus longs
    k = 2
    while True:
        # Générer les candidats à partir des motifs fréquents de la précédente étape
        candidates = generate_candidates(frequent_patterns)

        # Compter le support des candidats générés
        candidate_support = count_support(sequences, candidates)

        # Filtrer les motifs fréquents
        frequent_patterns = {pattern: support for pattern, support in candidate_support.items() if support >= min_support}

        if not frequent_patterns:
            break  # Arrêter si aucun nouveau motif fréquent n'est trouvé

        # Ajouter les nouveaux motifs fréquents aux résultats finaux
        all_frequent_patterns.update(frequent_patterns)
        k += 1

    return all_frequent_patterns


In [15]:
# Minimum support
min_support = 3

# Appliquer l'algorithme SPADE
frequent_patterns = spade_algorithm(sequences_dict_first, min_support)

# Afficher les motifs séquentiels fréquents
print("Motifs séquentiels fréquents trouvés par SPADE :")
for pattern, support in frequent_patterns.items():
    print(f"Motif : {pattern}, Support : {support}")


Motifs séquentiels fréquents trouvés par SPADE :
Motif : ('eggs',), Support : 103
Motif : ('meatballs',), Support : 13
Motif : ('burgers',), Support : 39
Motif : ('turkey',), Support : 37
Motif : ('avocado',), Support : 16
Motif : ('milk',), Support : 56
Motif : ('mineral water',), Support : 120
Motif : ('green tea',), Support : 58
Motif : ('whole wheat rice',), Support : 21
Motif : ('energy bar',), Support : 17
Motif : ('low fat yogurt',), Support : 31
Motif : ('whole wheat pasta',), Support : 16
Motif : ('french fries',), Support : 79
Motif : ('soup',), Support : 38
Motif : ('shallot',), Support : 7
Motif : ('light cream',), Support : 6
Motif : ('frozen vegetables',), Support : 43
Motif : ('spaghetti',), Support : 96
Motif : ('pet food',), Support : 5
Motif : ('cookies',), Support : 40
Motif : ('cooking oil',), Support : 24
Motif : ('champagne',), Support : 24
Motif : ('salmon',), Support : 22
Motif : ('shrimp',), Support : 38
Motif : ('chicken',), Support : 26
Motif : ('honey',), Su